<a id='top'></a>

# CSCI 3202, Fall 2020

# Friday Oct 30, 2020 

<br>


In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---


## Markov models - random walk to Taco Bell

Your friend Chris went to a party last night and left feeling a bit hungry.  He decides to head for Taco Bell. The party, is at the corner of 6th Street East and 3rd Street North, and Taco Bell, is located at the corner of 2nd Street East and 1st Street North.  A figure depicting this neighborhood is given below.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/random_walk_to_taco_bell.png" style="width: 650px;"/>

Chris' sense of direction quite poor, so at each intersection along his way, he picks any one of the available directions with equal probability.  Chris at least knows not to go north of 4th Street North, south of 0th Street North, east of 7th Street East, or west of 0th Street East, and has the common decency not to cut through anyone's yard (i.e., he only walks along streets).

Suppose Chris only cares about traveling between from one intersection to another, and considers one *move* to be walking one block, from one intersection to an adjacent intersection.

Since this grid is precisely a Cartesian coordinate grid, let the bottom-left corner of the neighborhood be represented by $(0,0)$.  Then the available moves from that location are to walk East (right, in the $+x$ direction) to $(1,0)$ or North to $(0,1)$.



### (a)

Create a class for `Neighborhood` and for `Agent`, to represent the neighborhood and the agent trying to get to Taco Bell:

`Neighborhood(n_northsouth, n_eastwest, taco_bell)`:
* `n_northsouth` and `n_eastwest` provide the number of streets running north-south and the number of streets running east-west, respectively.  In the given figure, there are 5 streets running east-west, for example.
* `taco_bell` is a tuple providing the coordinates of Taco Bell in this neighborhood.
* has attributes for:
  * number of streets running north-south
  * number of streets running east-west
  * all of the intersections in the neighborhood
  * the location of the Taco Bell in the neighborhood
* Implement in your code a check to make sure the location of the Taco Bell is within the neighborhood's coordinates.  Assume that the south-west corner of the neighborhood is always $(0,0)$.

`Agent(name, loc, neighborhood)`:
* In the constructor, provide the agent with a `name` (string), initial location as a tuple (`loc`), and a `Neighborhood` to live in. Store these as attributes.
* Fill in the rest of the needed methods for the agent:
  * `available_moves()` returns a list of tuples, representing the locations the agent can walk to from its current location
  * `random_move()` returns one of the possible moves
  * `walk(move)` updates the agent's location, if they make the given argument `move`
  * `at_taco_bell()` returns True if the agent is at the same intersection as the neighborhood Taco Bell, and False otherwise

In [425]:
class Neighborhood:
    def __init__(self, n_northsouth, n_eastwest, taco_bell):
        '''Set up the layout of the neighborhood by giving the # streets
        running North/South (n_northsouth) and the # streets running 
        East/West). Based on these, store all the available locations 
        (intersections) in the neighborhood, and make sure the given
        coordinates for Taco Bell are valid'''
        
        # your code goes here...
        
        # Solution:
        self.n_ns = n_northsouth
        self.n_ew = n_eastwest
        self.locations = [(col, row) for col in range(n_northsouth) for row in range(n_eastwest)]
        self.tb = taco_bell
        assert taco_bell in self.locations, 'Taco Bell must be within the neighborhood domain'
        
class Agent:
    def __init__(self, name, loc, neighborhood):
        
        # your code goes here...
        
        # Solution:
        self.name = name
        self.loc = loc
        self.nh = neighborhood
        
    def available_moves(self):
        '''Return a list of available intersections the agent can move to,
        based on the layout of the agent's neighborhood'''
        
        # your code goes here...
        
        # Solution:
        moves = []
        if self.loc[0] > 0:
            # can move West
            moves.append((self.loc[0]-1, self.loc[1]))
        if self.loc[0] < self.nh.n_ns-1:
            # can move East
            moves.append((self.loc[0]+1, self.loc[1]))
        if self.loc[1] > 0:
            # can move South
            moves.append((self.loc[0], self.loc[1]-1))
        if self.loc[1] < self.nh.n_ew-1:
            # can move North
            moves.append((self.loc[0], self.loc[1]+1))
        return moves

    def random_move(self):
        '''Return a random move out of the available moves
        from the agent`s current location'''
        
        # your code goes here...
        
        # Solution:
        moves = self.available_moves()
        n_moves = len(self.available_moves())
        return moves[np.random.randint(n_moves)]
    
    def walk(self, move):
        '''Update the agent to a new location'''
        
        # your code goes here...
        
        # Solution:
        self.loc = move
        
    def at_taco_bell(self):
        '''Return True if the agent is at Taco Bell, and False otherwise'''
        
        # your code goes here...
        
        # Solution:
        return self.loc==self.nh.tb
    
    def __repr__(self):
        return '{} at {}'.format(self.name, self.loc)


### (b)

Create a neighborhood and Chris agent to represent the situation above.

Then, run an ensemble of 1,000 simulations to obtain a sample for the number of blocks Chris must travel in order to arrive at Taco Bell at (2,1), starting from the party at (6,3).  Report the expected number of blocks traveled.

In [427]:
# Solution:

taco_bell = (2,1)
nh = Neighborhood(n_eastwest=5, n_northsouth=8, taco_bell=taco_bell)
n_sample = 1000
sample_blocks = []
for n in range(n_sample):
    chris = Agent('Chris', (6,3), nh)
    arrived = False
    blocks = 0
    while not arrived:
        chris.walk(chris.random_move())
        blocks += 1
        arrived = chris.at_taco_bell()
    sample_blocks.append(blocks)
    
print(np.mean(sample_blocks))

81.622


**Reflection:**  The sequence of states (coordinates) that Chris passed through in his travels is a **Markov chain** - each new state depended only on the previous one.  This process, called a **random walk**, can be a useful way to explore a state space.



### (c)

Let us explore one of the characteristics of a Markov chain that we often find ourselves interested in:  the **expected time of first return** to a state.  In particular, let's examine Chris' **expected time of first return to Taco Bell**.

Build an ensemble of 1,000 simulations of how many blocks Chris must travel to return to Taco Bell, given that he starts at Taco Bell in the neighborhood depicted in the figure above.

We can estimate the expected time of first return to Taco Bell using the **mean** number of blocks Chris must travel in order to make it back to those tasty tacos. Report this estimate based on your simulation results.

In [428]:
# Solution:

taco_bell = (2,1)
nh = Neighborhood(n_eastwest=5, n_northsouth=8, taco_bell=taco_bell)
n_sample = 1000
sample_blocks = []
for n in range(n_sample):
    chris = Agent('Chris', taco_bell, nh)
    arrived = False
    blocks = 0
    while not arrived:
        chris.walk(chris.random_move())
        blocks += 1
        arrived = chris.at_taco_bell()
    sample_blocks.append(blocks)

print(np.mean(sample_blocks))

31.638
